<a href="https://colab.research.google.com/github/koheikobayashi/pca-k-means/blob/main/pca_k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# overview

1. [colaboratoryでcsvを読み込む](https://colab.research.google.com/drive/1PkuqhusCcuD-a3t-t5Lw6MMjlNQ35g87#scrollTo=zYj45tt3wJYb&line=9&uniqifier=1)
2.[ファイルとモジュールのインポート](https://colab.research.google.com/drive/1PkuqhusCcuD-a3t-t5Lw6MMjlNQ35g87#scrollTo=zYj45tt3wJYb&line=3&uniqifier=1)
3.[相関係数を計算](https://colab.research.google.com/drive/1PkuqhusCcuD-a3t-t5Lw6MMjlNQ35g87#scrollTo=27EZUy7Mws_C&line=1&uniqifier=1)
4.主成分分析の実施
    
    4.1主成分分析を行う前に、各変数を標準化する

    4.2 sklearnのPCAクラスを用いて主成分分析を実施

    4.3 主成分から分析される主成分得点を計算

    4.4 主成分得点を用いて、k-means法によるクラスター分析を実施

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NPB2021.csv to NPB2021 (4).csv


In [ ]:
#データのインポート
import pandas as pd
import io
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.decomposition import PCA
import csv
import numpy as np
from sklearn.cluster import KMeans

NPB2021 = pd.read_csv(io.BytesIO(uploaded['NPB2021.csv']))
NPB2021.head()

,選 手,打率,試合,打席,打数,得点,安打,二塁打,三塁打,本塁打,...,盗塁刺,犠打,犠飛,四球,故意四,死球,三振,併殺打,長打率,出塁率
0,鈴木 誠也,0.317,132,533,435,77,138,26,0,38,...,4,0,5,87,11,6,88,7,0.639,0.433
1,坂倉 将吾,0.315,132,484,422,58,133,24,2,12,...,3,2,5,51,1,4,60,9,0.467,0.390
2,牧 秀悟,0.314,137,523,487,73,153,35,3,22,...,1,1,2,27,1,6,85,16,0.534,0.356
3,近本 光司,0.313,140,612,569,91,178,33,5,10,...,7,4,2,33,0,4,58,6,0.441,0.354
4,桑原 将志,0.310,135,571,519,84,161,39,2,14,...,7,4,0,44,1,4,75,12,0.474,0.369


In [ ]:
#相関係数を計算
NPB2021.corr()

,打率,試合,打席,打数,得点,安打,二塁打,三塁打,本塁打,塁打,...,盗塁刺,犠打,犠飛,四球,故意四,死球,三振,併殺打,長打率,出塁率
打率,1.000000,0.404662,0.521522,0.504948,0.593167,0.829029,0.534418,0.230080,0.018006,0.530745,...,0.335398,0.065215,0.128652,0.121910,-0.056657,0.061844,-0.445127,-0.086668,0.374126,0.670739
試合,0.404662,1.000000,0.814162,0.751858,0.536914,0.684128,0.386458,0.074681,0.254788,0.601101,...,0.186982,-0.202160,0.203957,0.334093,0.132267,0.242534,0.083266,0.210060,0.289427,0.387776
打席,0.521522,0.814162,1.000000,0.931006,0.662115,0.858267,0.440461,0.059889,0.327131,0.749847,...,0.224833,-0.232349,0.207910,0.402483,0.133254,0.214744,-0.141563,0.200376,0.371281,0.474949
打数,0.504948,0.751858,0.931006,1.000000,0.558454,0.899343,0.461471,0.218658,0.122100,0.632472,...,0.193378,-0.205688,0.110731,0.057988,0.016872,0.136554,-0.223647,0.218752,0.185987,0.201797
得点,0.593167,0.536914,0.662115,0.558454,1.000000,0.661870,0.524122,0.122410,0.522445,0.818727,...,0.372235,-0.181155,0.065326,0.411021,-0.102387,0.235380,0.171436,-0.202635,0.685067,0.604624
安打,0.829029,0.684128,0.858267,0.899343,0.661870,1.000000,0.579101,0.267599,0.075423,0.669908,...,0.305651,-0.096900,0.108332,0.087775,-0.027474,0.115831,-0.367254,0.088482,0.301126,0.462074
二塁打,0.534418,0.386458,0.440461,0.461471,0.524122,0.579101,1.000000,-0.240029,0.208539,0.579873,...,-0.066653,-0.316157,-0.160477,0.135557,0.002934,0.004513,-0.138566,0.246452,0.450324,0.390472
三塁打,0.230080,0.074681,0.059889,0.218658,0.122410,0.267599,-0.240029,1.000000,-0.503653,-0.194755,...,0.491175,0.392193,-0.284188,-0.470858,-0.299409,0.176297,-0.057727,-0.402338,-0.377942,-0.200799
本塁打,0.018006,0.254788,0.327131,0.122100,0.522445,0.075423,0.208539,-0.503653,1.000000,0.782768,...,-0.156464,-0.554701,0.305919,0.669721,0.377167,0.123101,0.472871,0.139289,0.909206,0.452958
塁打,0.530745,0.601101,0.749847,0.632472,0.818727,0.669908,0.579873,-0.194755,0.782768,1.000000,...,0.068138,-0.482401,0.235446,0.530024,0.245665,0.167926,0.148943,0.151299,0.875290,0.616252


In [ ]:
#各変数を標準化して比較
#(標準化された成績)={(元の成績-元の成績の平均)/元の成績の標準偏差}
NPB2021s = NPB2021.iloc[:, 1:].apply(lambda x: (x-x.mean())/x.std(), axis=0)
NPB2021s.head()

,打率,試合,打席,打数,得点,安打,二塁打,三塁打,本塁打,塁打,...,盗塁刺,犠打,犠飛,四球,故意四,死球,三振,併殺打,長打率,出塁率
0,1.683342,0.162839,0.250662,-0.596532,1.137619,0.432339,0.430357,-0.985277,2.031208,1.733666,...,0.389195,-0.571773,1.025305,2.103905,3.143272,0.725488,0.059501,-0.645436,2.712508,2.714251
1,1.600914,0.162839,-0.674134,-0.863942,-0.071598,0.200211,0.130978,0.031783,-0.349024,-0.141819,...,-0.025946,-0.146268,1.025305,0.270548,-0.595936,-0.061482,-0.892510,-0.248244,0.410327,1.406545
2,1.559700,0.717185,0.061928,0.473111,0.883047,1.128724,1.777563,0.540313,0.566450,1.316892,...,-0.856228,-0.359020,-0.671751,-0.951691,-0.595936,0.725488,-0.042500,1.141925,1.307107,0.372544
3,1.518485,1.049793,1.741661,2.159856,2.028621,2.289365,1.478184,1.557373,-0.532119,1.108504,...,1.634617,0.279238,-0.671751,-0.646131,-0.969857,-0.061482,-0.960511,-0.844031,0.062323,0.311721
4,1.394843,0.495447,0.967852,1.131353,1.583120,1.500129,2.376320,0.031783,-0.165930,0.992734,...,1.634617,0.279238,-1.803122,-0.085939,-0.595936,-0.061482,-0.382504,0.347542,0.504020,0.767897


In [ ]:
#主成分分析を実施
pca = PCA()
pca.fit(NPB2021s)

PCA()

In [ ]:
#主成分から算出される主成分得点を計算
feature = pca.transform(NPB2021s)
NPB2021pca= pd.DataFrame(feature, columns=["PC{}".format(x + 1) 
                               for x in range(len(NPB2021s.columns))])

In [ ]:
#この主成分得点を用いて、k-means法によるクラスター分析を実施
NPB2021pca_k = KMeans(n_clusters=3).fit_predict(NPB2021pca)
NPB2021pca_k

array([0, 1, 0, 2, 2, 0, 0, 1, 2, 1, 1, 1, 2, 0, 1, 1, 2, 2, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)